# Connecting to Neo4j


In [ ]:
from langchain.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt://44.203.87.67:7687",
    username="neo4j",
    password="november-bay-fillers"
)

r = graph.query("MATCH (m:Movie{title: 'Toy Story'}) RETURN m")
print(r)

In [ ]:
print(graph.schema)

In [ ]:
# REFRESHING THE SCHEMA

# You can refresh the schema by calling the graph.refresh_schema() method.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

from langchain.vectorstores.neo4j_vector import Neo4jVector

inference_api_key = os.getenv("INFERENCE_API_KEY", None)

embedding_provider = HuggingFaceInferenceAPIEmbeddings(
    api_key=inference_api_key, model_name="sentence-transformers/all-MiniLM-l6-v2"
)

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    url="bolt://44.203.87.67:7687",
    username="neo4j",
    password="november-bay-fillers",
    index_name="moviePlots",
    embedding_node_property="embedding",
    text_node_property="plot",
)

r = movie_plot_vector.similarity_search("A movie where aliens land and attack earth.")
print(r)

# Creating a new vector index

In [1]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

/Users/charanahu/VS_Code/GitHub/knowledge_graph/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.schema import Document

text = """Nuclear power in space is the use of nuclear power in outer space, typically either small fission systems or radioactive decay for electricity or heat. Another use is for scientific observation, as in a Mössbauer spectrometer. The most common type is a radioisotope thermoelectric generator, which has been used on many space probes and on crewed lunar missions. Small fission reactors for Earth observation satellites, such as the TOPAZ nuclear reactor, have also been flown.[1] A radioisotope heater unit is powered by radioactive decay and can keep components from becoming too cold to function, potentially over a span of decades.[2]

The United States tested the SNAP-10A nuclear reactor in space for 43 days in 1965,[3] with the next test of a nuclear reactor power system intended for space use occurring on 13 September 2012 with the Demonstration Using Flattop Fission (DUFF) test of the Kilopower reactor.[4]

After a ground-based test of the experimental 1965 Romashka reactor, which used uranium and direct thermoelectric conversion to electricity,[5] the USSR sent about 40 nuclear-electric satellites into space, mostly powered by the BES-5 reactor. The more powerful TOPAZ-II reactor produced 10 kilowatts of electricity.[3]

Examples of concepts that use nuclear power for space propulsion systems include the nuclear electric rocket (nuclear powered ion thruster(s)), the radioisotope rocket, and radioisotope electric propulsion (REP).[6] One of the more explored concepts is the nuclear thermal rocket, which was ground tested in the NERVA program. Nuclear pulse propulsion was the subject of Project Orion.[7]

Regulation and hazard prevention[edit]
After the ban of nuclear weapons in space by the Outer Space Treaty in 1967, nuclear power has been discussed at least since 1972 as a sensitive issue by states.[8] Particularly its potential hazards to Earth's environment and thus also humans has prompted states to adopt in the U.N. General Assembly the Principles Relevant to the Use of Nuclear Power Sources in Outer Space (1992), particularly introducing safety principles for launches and to manage their traffic.[8]

Benefits

Both the Viking 1 and Viking 2 landers used RTGs for power on the surface of Mars. (Viking launch vehicle pictured)
While solar power is much more commonly used, nuclear power can offer advantages in some areas. Solar cells, although efficient, can only supply energy to spacecraft in orbits where the solar flux is sufficiently high, such as low Earth orbit and interplanetary destinations close enough to the Sun. Unlike solar cells, nuclear power systems function independently of sunlight, which is necessary for deep space exploration. Nuclear-based systems can have less mass than solar cells of equivalent power, allowing more compact spacecraft that are easier to orient and direct in space. In the case of crewed spaceflight, nuclear power concepts that can power both life support and propulsion systems may reduce both cost and flight time.[9]

Selected applications and/or technologies for space include:

Radioisotope thermoelectric generator
Radioisotope heater unit
Radioisotope piezoelectric generator
Radioisotope rocket
Nuclear thermal rocket
Nuclear pulse propulsion
Nuclear electric rocket
"""

docs = [
    Document(
        page_content=split,
        metadata={"source": "https://en.wikipedia.org/wiki/Nuclear_power_in_space"},
    )
    for split in text.split()
]

In [3]:
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.schema import Document

# A list of Documents
documents = docs

In [4]:


# Service used to create the embeddings
embedding_provider = embeddings

In [5]:
new_vector_1 = Neo4jVector.from_documents(
    documents,
    embedding_provider,
    url="bolt://44.203.87.67:7687",
    username="neo4j",
    password="november-bay-fillers",
    index_name="myVectorIndexes",
    node_label="Chunk",
    text_node_property="text",
    embedding_node_property="embedding",
    create_id_index=True,
)

ValueError: Index with name myVectorIndex already exists.The provided embedding function and vector index dimensions do not match.
Embedding function dimension: 768
Vector index dimension: 384

In [ ]:
graph.refresh_schema()

# Creating a Retriever chain

In [ ]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool
load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY", None)
llm = ChatGoogleGenerativeAI(
    model="gemini-pro", convert_system_message_to_human=True
)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.vectorstores.neo4j_vector import Neo4jVector

chat_llm = llm

embedding_provider = embeddings

movie_plot_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    url="bolt://44.203.87.67:7687",
    username="neo4j",
    password="november-bay-fillers",
    index_name="myVectorIndex2",
    embedding_node_property="embedding",
    text_node_property="plot",
)

retrievalQA = RetrievalQA.from_llm(
    llm=chat_llm,
    retriever=movie_plot_vector.as_retriever()
)

r = retrievalQA("A mission to the moon goes wrong")
print(r)

In [ ]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=inference_api_key, model_name="sentence-transformers/all-MiniLM-l6-v2"
)

query_result = embeddings.embed_query("hello")
query_result[:3]